<a href="https://colab.research.google.com/github/AndresMontesDeOca/Redes_Neuronales/blob/main/06_Balance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Multiperceptrón
---
**Montar la carpeta de Google Drive y definir constantes para trabajar**

In [15]:
ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/Redes_Neuronales/Fuentes/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/Redes_Neuronales/Data/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


### Carga e Inspección de Datos

In [16]:
# pip install ucimlrepo

In [17]:


from ucimlrepo import fetch_ucirepo

# fetch dataset
balance_scale = fetch_ucirepo(id=12)

# data (as pandas dataframes)
X = balance_scale.data.features
y = balance_scale.data.targets


{'uci_id': 12, 'name': 'Balance Scale', 'repository_url': 'https://archive.ics.uci.edu/dataset/12/balance+scale', 'data_url': 'https://archive.ics.uci.edu/static/public/12/data.csv', 'abstract': 'Balance scale weight & distance database', 'area': 'Social Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 625, 'num_features': 4, 'feature_types': ['Categorical'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1976, 'last_updated': 'Fri Feb 09 2024', 'dataset_doi': '10.24432/C5488X', 'creators': ['R. Siegler'], 'intro_paper': None, 'additional_info': {'summary': 'This data set was generated to model psychological experimental results.  Each example is classified as having the balance scale tip to the right, tip to the left, or be balanced.  The attributes are the left weight, the left distance, the right weight, and the right distance.  The corr

In [19]:
import pandas as pd      # para trabajar con archivos de datos csv, excel
import numpy as np

from sklearn import preprocessing, model_selection
from sklearn.neural_network import MLPClassifier
from matplotlib import pyplot as plt

# nombre_archivo = DATOS_DIR + 'Balance.csv' # archivo de hojas

# df = pd.read_csv(nombre_archivo)

df = pd.concat([X, y], axis=1)

# Para hacer una inspección rápida de los datos
display(df.head(12))
df.to_csv('Balance.csv')

# #print(df.describe())
# df[' Balance'].hist(bins=3)

,right-distance,right-weight,left-distance,left-weight,class
0,1,1,1,1,B
1,2,1,1,1,R
2,3,1,1,1,R
3,4,1,1,1,R
4,5,1,1,1,R
5,1,2,1,1,R
6,2,2,1,1,R
7,3,2,1,1,R
8,4,2,1,1,R
9,5,2,1,1,R


### Selección de atributos y target

In [20]:
# %% separa atributos y clases
X_raw = df.iloc[:,0:-1]  # recupera todas las columnas salvo la primera (es la clase)
Y_raw = df.iloc[:,-1]    # recupera solo la última columna (es la clase)

# Binarizador para convertir el nombre de la clase en one hot encoding
binarizer = preprocessing.LabelBinarizer()

# Binariza cada clase como una combinación de un 1 y 0s
Y_raw = binarizer.fit_transform(Y_raw)
# Y_raw==pd.get_dummies(df[' Balance']).to_numpy() # forma alternativa para codificar

print('Las clases del dataset son :', binarizer.classes_)
print(Y_raw)

# %% Separa ejemplos para enternamiento y testeo
TEST_SIZE = 0.3 # proporcion entre testeo entre entrenamiento y testeo
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X_raw, Y_raw, test_size=TEST_SIZE)#, random_state=42)
print('\nDatos de Entrenamiento: %d   Datos de Testeo: %d' % (len(Y_train), len(Y_test) ))

# Escala los atributos de los ejemplo
scaler = preprocessing.StandardScaler()
#scaler = preprocessing.MinMaxScaler()

X_train  = scaler.fit_transform( X_train )
X_test   = scaler.transform( X_test )

Las clases del dataset son : ['B' 'L' 'R']
[[1 0 0]
 [0 0 1]
 [0 0 1]
 ...
 [0 1 0]
 [0 1 0]
 [1 0 0]]

Datos de Entrenamiento: 437   Datos de Testeo: 188


### Entrenamiento y Evaluación del Modelo

In [21]:
FunH = 'tanh'   # opciones: identity logistic tanh relu
ocultas = (8, 4)

alfa = 0.005
CotaError = 1.0e-4
MAX_ITE = 2500

modelo = MLPClassifier(max_iter=MAX_ITE, hidden_layer_sizes=ocultas, alpha=alfa,
                       solver='adam', activation=FunH, tol=CotaError,
                       verbose=False).fit(X_train, Y_train)


#  ########### Medición del entrenamiento ######################
Y_pred = modelo.predict(X_train)
score = modelo.score(X_train, Y_train)

# "invierte" la transformacion binaria para obtener los nombres de las clases
Y_it = binarizer.inverse_transform(Y_train)
Y_pred_it = binarizer.inverse_transform(Y_pred)

# calculo manual del accuracy
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
print('      Score: %6.2f' % (score) )


Efectividad:  99.54%
      Score:   0.99


### Evaluación del Modelo con Datos de Prueba

In [22]:
#  ########### Medición del testeo ######################
Y_pred = modelo.predict(X_test)
score = modelo.score(X_test, Y_test)

# "invierte" la transformacion binaria para obtener los nombres de las clases
Y_it = binarizer.inverse_transform(Y_test)
Y_pred_it = binarizer.inverse_transform(Y_pred)

# calculo manual del accuracy
print('Efectividad: %6.2f%%' % (100*(Y_pred_it == Y_it).sum()/len(Y_it)) )
print('      Score: %6.2f' % (score) )

Efectividad:  97.34%
      Score:   0.96
